In [0]:
curropted_record_df = spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","true")\
            .option("mode","permissive")\
            .load("/FileStore/tables/curropted_record.csv")
curropted_record_df.show()

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|   India|
|  4|Prantosh| 17|200000|     Kolkata|   India|
|  5|  Vikash| 31|300000|        null|nominee5|
+---+--------+---+------+------------+--------+



In [0]:
curropted_record_df = spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","true")\
            .option("mode","dropmalformed")\
            .load("/FileStore/tables/curropted_record.csv")
curropted_record_df.show()

+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        null|nominee5|
+---+------+---+------+------------+--------+



In [0]:
curropted_record_df = spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","true")\
            .option("mode","failfast")\
            .load("/FileStore/tables/curropted_record.csv")
curropted_record_df.show()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-892831296662660>:6
      1 curropted_record_df = spark.read.format("csv")\
      2             .option("header","true")\
      3             .option("inferschema","true")\
      4             .option("mode","failfast")\
      5             .load("/FileStore/tables/curropted_record.csv")
----> 6 curropted_record_df.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeE

In [0]:
#creating manual schema
from pyspark.sql.types import StructField, StructType, IntegerType, StringType
curropted_record_schema = StructType([
                                        StructField("id",IntegerType(),True),
                                        StructField("name",StringType(),True),
                                        StructField("age",IntegerType(),True),
                                        StructField("salary",IntegerType(),True),
                                        StructField("address",StringType(),True),
                                        StructField("nominee",StringType(),True),
                                        StructField("_corrupt_record",StringType(),True)
])

In [0]:
#printing records using manual schema
curropted_record_df = spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","true")\
            .option("mode","PERMISSIVE")\
            .schema(curropted_record_schema)\
            .load("/FileStore/tables/curropted_record.csv")
curropted_record_df.show(truncate = False)

+---+--------+---+------+------------+--------+-------------------------------------------+
|id |name    |age|salary|address     |nominee |_corrupt_record                            |
+---+--------+---+------+------------+--------+-------------------------------------------+
|1  |Manish  |26 |75000 |bihar       |nominee1|null                                       |
|2  |Nikita  |23 |100000|uttarpradesh|nominee2|null                                       |
|3  |Pritam  |22 |150000|Bangalore   |India   |3,Pritam,22,150000,Bangalore,India,nominee3|
|4  |Prantosh|17 |200000|Kolkata     |India   |4,Prantosh,17,200000,Kolkata,India,nominee4|
|5  |Vikash  |31 |300000|null        |nominee5|null                                       |
+---+--------+---+------+------------+--------+-------------------------------------------+



In [0]:
#creating seperate file for curropted records
curropted_record_df = spark.read.format("csv")\
            .option("header","true")\
            .option("inferschema","true")\
            .schema(curropted_record_schema)\
            .option("badRecordsPath","/FileStore/tables/bad_records")\
            .load("/FileStore/tables/curropted_record.csv")
curropted_record_df.show(truncate = False)

+---+--------+---+------+---------+-------+-----------------+
|id |name    |age|salary|address  |nominee|_curropted_Record|
+---+--------+---+------+---------+-------+-----------------+
|3  |Pritam  |22 |150000|Bangalore|India  |nominee3         |
|4  |Prantosh|17 |200000|Kolkata  |India  |nominee4         |
+---+--------+---+------+---------+-------+-----------------+



In [0]:
%fs
ls /FileStore/tables/bad_records/20240821T173550/bad_records/part-00000-e92de0e3-731f-423b-a89b-16fe57b91aa4

path,name,size,modificationTime
dbfs:/FileStore/tables/bad_records/20240821T173550/bad_records/part-00000-e92de0e3-731f-423b-a89b-16fe57b91aa4,part-00000-e92de0e3-731f-423b-a89b-16fe57b91aa4,692,1724261752000


In [0]:
#making dataframe to print the bad data
bad_data = spark.read.format("json")\
            .load("/FileStore/tables/bad_records/20240821T173550/bad_records/")
bad_data.show(truncate=False)

+-------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+----------------------------------------+
|path                                       |reason                                                                                                                       |record                                  |
+-------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------+----------------------------------------+
|dbfs:/FileStore/tables/curropted_record.csv|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 1,Manish,26,75000,bihar,nominee1        |1,Manish,26,75000,bihar,nominee1        |
|dbfs:/FileStore/tables/curropted_record.csv|org.apache.spark.SparkRuntimeException: [MALFORMED_CSV_RECORD] Malformed CSV record: 2,Nikita,23,100000